<a href="https://colab.research.google.com/github/arsengir/colab/blob/main/human_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/datasets/abdelrahmankhalil/100000-uk-used-car-data-set

https://drive.google.com/file/d/17xk2fS4QevT2NKziTD2puH9w9B8qXpCZ/view?usp=sharing

In [ ]:
! gdown --id 17xk2fS4QevT2NKziTD2puH9w9B8qXpCZ

In [ ]:
! unzip /content/uk_used_cars.zip -d /content/uk_used_cars

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/uk_used_cars/bmw.csv')

In [ ]:
df.head()

In [ ]:
len(df)

# Обучающая и тестовая выборка (train and test)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(df)

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
len(train) + len(test)

In [ ]:
len(test)/len(df)

In [ ]:
train, test = train_test_split(df, train_size=0.7)

In [ ]:
len(train)

In [ ]:
len(test)/len(df)

In [ ]:
train, test = train_test_split(df)

In [ ]:
len(test)/len(df)

In [ ]:
train, test = train_test_split(df,random_state=42)

In [ ]:
train

# Анализ данных

In [ ]:
train.isna().mean()

In [ ]:
df.isna().mean()

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [ ]:
train['price'].hist()

In [ ]:
train['price'].describe()

In [ ]:
train.groupby('year')['price'].agg(['count', 'mean', 'median'])

In [ ]:
train.groupby('year')['price'].median().plot()

# Baseline

In [ ]:
train['price'].describe()

In [ ]:
train['price'].mean()

In [ ]:
train['price_pred_mean'] = train['price'].mean()

In [ ]:
train

In [ ]:
train['err'] = train['price_pred_mean'] - train['price']

In [ ]:
train

In [ ]:
train['err_abs'] = abs(train['err'])

In [ ]:
train

In [ ]:
train['err_abs'].mean()

In [ ]:
train['err_abs_pct'] = train['err_abs']/train['price']

In [ ]:
train['err_abs_pct'].mean()

In [ ]:
train['price_pred_median'] = train['price'].median()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [ ]:
mean_absolute_error(train['price'], train['price_pred_mean'])

In [ ]:
mean_absolute_percentage_error(train['price'], train['price_pred_mean'])

In [ ]:
mean_absolute_error(train['price'], train['price_pred_median'])

In [ ]:
mean_absolute_percentage_error(train['price'], train['price_pred_median'])

In [ ]:
test['price_pred_median'] = train['price'].median()

In [ ]:
test

In [ ]:
mean_absolute_error(test['price'], test['price_pred_median'])

In [ ]:
mean_absolute_percentage_error(test['price'], test['price_pred_median'])

# Алгоритм на основе года

In [ ]:
train.groupby('year')['price'].agg(['count', 'mean', 'median'])

In [ ]:
model_year_mean = train.groupby('year')['price'].mean().reset_index()

In [ ]:
model_year_mean.columns

In [ ]:
model_year_mean.columns = ['year', 'price_pred_year_mean']

In [ ]:
model_year_mean

In [ ]:
train = train.merge(model_year_mean, how='left', on='year')

In [ ]:
mean_absolute_error(train['price'], train['price_pred_year_mean'])

In [ ]:
mean_absolute_percentage_error(train['price'], train['price_pred_year_mean'])

In [ ]:
test = test.merge(model_year_mean, how='left', on='year')

In [ ]:
test.isna().mean()

In [ ]:
mean_absolute_error(test['price'], test['price_pred_year_mean'])

In [ ]:
mean_absolute_percentage_error(test['price'], test['price_pred_year_mean'])

# Медиана по году

In [ ]:
model_year_median = train.groupby('year')['price'].median().reset_index()

In [ ]:
model_year_median = model_year_median.rename({'price':'price_pred_year_median'}, axis=1)

In [ ]:
model_year_median

In [ ]:
train = train.merge(model_year_median, how='left', on='year')

In [ ]:
mean_absolute_error(train['price'], train['price_pred_year_median'])

In [ ]:
mean_absolute_percentage_error(train['price'], train['price_pred_year_median'])

In [ ]:
test = test.merge(model_year_median, how='left', on='year')

In [ ]:
test.isna().mean()

In [ ]:
mean_absolute_error(test['price'], test['price_pred_year_median'])

In [ ]:
mean_absolute_percentage_error(test['price'], test['price_pred_year_median'])

# Добавляем еще одну переменную

In [ ]:
train

In [ ]:
train.groupby('engineSize')['price','year'].agg(['count', 'mean', 'median'])

In [ ]:
train.groupby('engineSize')['price'].agg(['count', 'mean', 'median'])

In [ ]:
def engine_group(x):
  if x <= 1.5:
    return '<=1.5'
  if x <= 2:
    return '<=2'
  return '>2'

In [ ]:
train['engine_group'] = train['engineSize'].apply(engine_group)

In [ ]:
train

In [ ]:
t = train.groupby('engine_group')['price'].agg(['count', 'mean', 'median'])

In [ ]:
t['count'].sum()

In [ ]:
train['engine_group2'] = pd.cut(train['engineSize'], [-float('inf'),1.5,2,float('inf')])

In [ ]:
t

In [ ]:
train.groupby('engine_group2')['price'].agg(['count', 'mean', 'median'])

In [ ]:
model_year_engine_median = train.groupby(['year','engine_group'])['price'].median().reset_index()

In [ ]:
model_year_engine_median

In [ ]:
train.groupby(['year','engine_group'])['price'].agg(['count', 'median'])

In [ ]:
t = train.pivot_table(index='year',columns='engine_group', values='price', aggfunc='median')

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(t)

In [ ]:
model_year_engine_median = model_year_engine_median.rename({'price':'price_pred_year_engine_median'}, axis=1)

In [ ]:
train = train.merge(model_year_engine_median, how='left', on=['year', 'engine_group'])

In [ ]:
mean_absolute_error(train['price'], train['price_pred_year_median'])

In [ ]:
mean_absolute_percentage_error(train['price'], train['price_pred_year_median'])

In [ ]:
mean_absolute_error(train['price'], train['price_pred_year_engine_median'])

In [ ]:
mean_absolute_percentage_error(train['price'], train['price_pred_year_engine_median'])

In [ ]:
test['engine_group'] = test['engineSize'].apply(engine_group)

In [ ]:
test.isna().mean()

In [ ]:
test = test.merge(model_year_engine_median, how='left', on=['year', 'engine_group'])

In [ ]:
test.isna().mean()

In [ ]:
test[test['price_pred_year_engine_median'].isna()]

In [ ]:
def error(y_true, y_pred):
    print(mean_absolute_error(y_true, y_pred))
    print(mean_absolute_percentage_error(y_true, y_pred))

In [ ]:
error(train['price'], train['price_pred_year_median'])

In [ ]:
error(test['price'], test['price_pred_year_median'])

In [ ]:
error(train['price'], train['price_pred_year_engine_median'])

In [ ]:
test_no_nan = test.dropna().copy()

In [ ]:
error(test_no_nan['price'], test_no_nan['price_pred_year_engine_median'])

# Улучшение модели

In [ ]:
train

In [ ]:
train.groupby('transmission')['price'].agg(['count', 'mean', 'median'])

In [ ]:
len(model_year_engine_median)

In [ ]:
model_year_engine_median

In [ ]:
train.groupby('year')['price'].agg(['count', 'median'])

In [ ]:
train['year_group'] = pd.cut(train['year'], [0,2010,2012,2014,2016,2018,float('inf')])

In [ ]:
train.groupby('year_group')['price'].agg(['count', 'median'])

In [ ]:
train.groupby(['year_group', 'engine_group'])['price'].agg(['count', 'median'])

In [ ]:
model_year_engine_group_median = train.groupby(['year_group', 'engine_group'])['price'].median().reset_index()

In [ ]:
model_year_engine_group_median

In [ ]:
model_year_engine_group_median = model_year_engine_group_median.rename({'price':'pred_price_year_engine_group_median'},axis=1)

In [ ]:
train = train.merge(model_year_engine_group_median, how='left', on=['year_group', 'engine_group'])

In [ ]:
error(train['price'], train['price_pred_year_engine_median'])

In [ ]:
error(train['price'], train['pred_price_year_engine_group_median'])

In [ ]:
model_year_engine_trn_group_median = (train.groupby(['year_group', 
                                                     'engine_group', 
                                                     'transmission'])
                                            ['price'].median().reset_index())

In [ ]:
model_year_engine_trn_group_median = model_year_engine_trn_group_median.rename({'price':'pred_price_year_engine_trn_median'}, axis=1)

In [ ]:
train = train.merge(model_year_engine_trn_group_median,how='left',on=['year_group', 'engine_group', 'transmission'])

In [ ]:
error(train['price'], train['price_pred_year_engine_median'])

In [ ]:
error(train['price'], train['pred_price_year_engine_group_median'])

In [ ]:
error(train['price'], train['pred_price_year_engine_trn_median'])

In [ ]:
test['year_group'] = pd.cut(test['year'], [0,2010,2012,2014,2016,2018,float('inf')])

In [ ]:
test = test.merge(model_year_engine_trn_group_median,how='left',on=['year_group', 'engine_group', 'transmission'])

In [ ]:
test.isna().mean()

In [ ]:
test_no_nan = test.dropna().copy()

In [ ]:
error(train['price'], train['pred_price_year_engine_trn_median'])

In [ ]:
error(test_no_nan['price'], test_no_nan['pred_price_year_engine_trn_median'])

In [ ]:
error(test_no_nan['price'], test_no_nan['price_pred_year_engine_median'])